# Pointcloud Tree Analysis
An experimental notebook

We will compare and analyse isolated trees using three different point cloud sources:
1. _[AHN](https://www.ahn.nl) data_ (publicly available)
2. _Cyclo Media_
3. _Sonarski_

A visual impression of what the an isolated three looks like in the three datasets. 
This particular tree is located at 1219.13, 4874.34 (LTR: AHN, CycloMedia, Sonarski):

![Comparison of datasets (side-view)](../imgs/20_side.png)
![Comparison of datasets (top-view)](../imgs/20_top.png)

#### Import Modules

In [1]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [2]:
# Import modules.
import numpy as np
import open3d as o3d

import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.las_utils as las_utils

#### Load datasets

In [3]:
ahn_tree_path = '../datasets/single_selection/single_121913_487434_AHN.las'
cyclo_tree_path = '../datasets/single_selection/single_121913_487434_Cyclo.las'
sonarski_tree_path = '../datasets/single_selection/single_121913_487434_Sonarski.las'

In [4]:
las_files = {
    'AHN': las_utils.read_las(ahn_tree_path), 
    'CycloMedia': las_utils.read_las(cyclo_tree_path),
    'Sonarski': las_utils.read_las(sonarski_tree_path)
}

## 1. Dataset statistics

In [10]:
las_utils.print_statistics(las_files['CycloMedia'])

Pointcloud Statistics:
------------------------
No. points: 168265
Mean distance: 0.04 m
x: 12.86 m [121906.94 121919.8 ]
y: 12.6 m [487427.54 487440.14]
z: 15.27 m [ 1.71 16.98]



#### Open3D experiments

In [5]:
cyclo_pcd = las_utils.to_o3d(las_files['CycloMedia'])
cyclo_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamRadius(.15))
cyclo_pcd

PointCloud with 168265 points.

In [15]:
# Outlier Selection
def outlier_removal(pcd):
    print("Radius oulier removal")
    cl, ind = pcd.remove_radius_outlier(nb_points=2, radius=0.3)

    inlier_cloud = pcd.select_by_index(ind)
    outlier_cloud = pcd.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

outlier_removal(cyclo_pcd)

Radius oulier removal


In [17]:
# Convex Hull
def pcd_convex_hull(pcd):
    hull, _ = pcd.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((1, 0, 0))
    o3d.visualization.draw_geometries([pcd, hull_ls])

pcd_convex_hull(cyclo_pcd)

![Convex Hull](../imgs/convex_hull.png)

In [ ]:
# Poisson Surface Reconstruction
def poisson_sr(pcd):
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)
    o3d.visualization.draw_geometries([mesh])

poisson_sr(cyclo_pcd)